### Experiment Vectorization Approach

In [1]:
####imports section####
from IPython.display import Markdown, display
import tqdm as notebook_tqdm
import os
import warnings
import json
from getpass import getpass
from langchain_community.document_loaders import PyPDFLoader
#Imports for RAG Pipeline and Document Store
from haystack import Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.rankers.transformers_similarity import TransformersSimilarityRanker
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
#Imports for Chat Approach
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.dataclasses import ChatMessage
from haystack.components.generators.utils import print_streaming_chunk
from haystack import Pipeline
import gradio as gr

warnings.filterwarnings("ignore")

### Load Data

In [2]:
loader = PyPDFLoader("./downloads/Clinch River Breeder Reactor Project Preliminary Safety Analysis Report, Volume 3. - ML082960494.pdf")
pages = loader.load_and_split()
print("Size of DOC: \n", len(pages))

Size of DOC: 
 0


### Documentation Store
-> Will store the content and the respective embeddings

In [3]:
#size of documents
range_pages = range(0,len(pages))

#Define a DocStore 
document_store = InMemoryDocumentStore()

#Convert to Haystack Docs
docs = [Document(content=pages[i].page_content, meta=pages[i].metadata) for i in range_pages]

#Embedd documents
doc_embedder = SentenceTransformersDocumentEmbedder(model="sentence-transformers/all-MiniLM-L6-v2", normalize_embeddings=True)
doc_embedder.warm_up()

docs_with_embeddings = doc_embedder.run(docs)

print("Embedding size: ", len(docs_with_embeddings['documents'][0].embedding))

#Write documents -> Ids, Content and embeddings
document_store.write_documents(docs_with_embeddings["documents"])

Batches: 0it [00:00, ?it/s]

IndexError: list index out of range

### Build a RAG Pipeline
- To define a RAG Pipeline we need:
    - A Text Embedder, for embedding user queries, these embeddings will be use to compare to the embedding documents;
    - A Retriever, task to to retrieve the documents that match the user query;
    - Template Prompt, it provides the context;
    - Generator, LLM Model;

In [4]:
template = """
Given the following information, answer the question.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""
#Define pipeline components
pipe_comp = [
    ('text_embedder', SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2", normalize_embeddings =True)),
    ('retriever', InMemoryEmbeddingRetriever(document_store=document_store, scale_score = True)),
    # ('ranker', TransformersSimilarityRanker(model='cross-encoder/ms-marco-MiniLM-L-6-v2', top_k=25)),
    ('prompt_builder', PromptBuilder(template=template)),
    ('llm', OpenAIGenerator(model="gpt-3.5-turbo", generation_kwargs = {'temperature': 0.1}))
]

#Define pipeline
basic_rag_pipeline = Pipeline()

#1. Add components
for comp in pipe_comp:
    basic_rag_pipeline.add_component(*comp)


# #2. Connect components
# #LOGIC: Output -> Input
basic_rag_pipeline.connect('text_embedder.embedding', "retriever.query_embedding")
basic_rag_pipeline.connect('retriever', 'prompt_builder.documents')
basic_rag_pipeline.connect('prompt_builder', 'llm')


# #With Ranker
# basic_rag_pipeline.connect('text_embedder', "retriever")
# basic_rag_pipeline.connect('retriever', "ranker")
# basic_rag_pipeline.connect('ranker', 'prompt_builder')
# basic_rag_pipeline.connect('prompt_builder', 'llm')


🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

### Ask Phase

In [5]:
question = "What are the conditions for natural gas be considrered a sustainable activity?"

query = {
    'text_embedder': {
        'text': question
    },

    'prompt_builder': {
        'question': question
    }
}


response = basic_rag_pipeline.run(query)
Markdown(response['llm']['replies'][0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Based on the information provided, natural gas would be considered a sustainable activity if it meets the following criteria:

1. Climate change mitigation: The direct greenhouse gas emissions of the activity are lower than 270 gCO2e/kWh.
2. Climate change adaptation: The activity complies with the criteria set out in the relevant appendix.
3. Sustainable use and protection of water and marine resources: The activity complies with the criteria set out in the relevant appendix.
4. Pollution prevention and control: The activity complies with the criteria set out in the relevant appendix.
5. Protection and restoration of biodiversity and ecosystems: The activity complies with the criteria set out in the relevant appendix.

Additionally, for natural gas to be considered sustainable, it should not cause significant harm to the environment and should contribute substantially to the transition to a circular economy.

### Chat Approach

In [7]:
#Convert HayStack Pipeline into a Tool
def rag_pipeline_func(query: str):
    result = basic_rag_pipeline.run(
        {
            'text_embedder': {'text': query},
            'prompt_builder': {'question': query}
        }
    )

    return {"reply": result['llm']["replies"][0]}


#Add to tools
tools = [
    {
        'type': 'function',
        'function': {
            'name': 'rag_pipeline_func',
            'description': 'Get information about the documents provided',
            'parameters': {
                'type': 'object',
                'properties': {
                    'query': {
                        'type': 'string',
                        'description': "The query to use in the search. Infer this from the user's message. It should be a question or a statement",
                    }
                },
                'required': ["query"],
            }
        },
    }
]


messages = [
    ChatMessage.from_system("Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."),
    ChatMessage.from_user("Can you provide description of how activities are considered sustainable?")

]

chat_generator = OpenAIChatGenerator(model="gpt-3.5-turbo", streaming_callback=print_streaming_chunk, generation_kwargs={'temperature': 0.1})
response = chat_generator.run(messages=messages, generation_kwargs={"tools": tools})

In [8]:
#Parse function calling information
function_call = json.loads(response["replies"][0].content)[0]
function_name = function_call["function"]["name"]
function_args = json.loads(function_call["function"]["arguments"])
print("Function Name:", function_name)
print("Function Arguments:", function_args)
# Find the correspoding function and call it with the given arguments
available_functions = {"rag_pipeline_func": rag_pipeline_func}
function_to_call = available_functions[function_name]
function_response = function_to_call(**function_args)
print("Function Response:", function_response)

Function Name: rag_pipeline_func
Function Arguments: {'query': 'How are activities considered sustainable?'}


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Function Response: {'reply': 'Activities are considered sustainable if they comply with specific criteria and guidelines outlined in the provided information. This includes measures such as transitioning to a circular economy, protecting and restoring biodiversity and ecosystems, mitigating climate change, adapting to climate change, and preventing pollution. Sustainable activities also involve actions such as reducing waste generation, promoting the reuse and recycling of materials, managing hazardous waste properly, minimizing the use of pesticides and fertilizers, and implementing environmental management systems. Additionally, sustainable activities contribute to the protection and restoration of habitats, ecosystems, and species, and ensure that they do not have significant negative impacts on the environment.'}


In [9]:
function_message = ChatMessage.from_function(content=json.dumps(function_response), name=function_name)
messages.append(function_message)
response = chat_generator.run(messages=messages, generation_kwargs={"tools": tools})

Based on the information provided, activities are considered sustainable if they comply with specific criteria and guidelines. This includes measures such as transitioning to a circular economy, protecting and restoring biodiversity and ecosystems, mitigating climate change, adapting to climate change, and preventing pollution. Sustainable activities also involve actions such as reducing waste generation, promoting the reuse and recycling of materials, managing hazardous waste properly, minimizing the use of pesticides and fertilizers, and implementing environmental management systems. Additionally, sustainable activities contribute to the protection and restoration of habitats, ecosystems, and species, and ensure that they do not have significant negative impacts on the environment.

### Chat APP Sim

In [11]:
#CHAT APP
chat_generator = OpenAIChatGenerator(model="gpt-3.5-turbo", generation_kwargs={'temperature': 0.1})
response = None
messages = [
    ChatMessage.from_system(
        "Present as much relevant information as possible, be descriptive."
    )
]

def chatbot_with_fc(message, history):
    messages.append(ChatMessage.from_user(message))
    response = chat_generator.run(messages=messages, generation_kwargs={"tools": tools})

    while True:
        # if OpenAI response is a tool call
        if response and response["replies"][0].meta["finish_reason"] == "tool_calls":
            function_calls = json.loads(response["replies"][0].content)
            print(response["replies"][0])
            for function_call in function_calls:
                ## Parse function calling information
                function_name = function_call["function"]["name"]
                function_args = json.loads(function_call["function"]["arguments"])

                ## Find the correspoding function and call it with the given arguments
                function_to_call = available_functions[function_name]
                function_response = function_to_call(**function_args)

                ## Append function response to the messages list using `ChatMessage.from_function`
                messages.append(ChatMessage.from_function(content=json.dumps(function_response), name=function_name))
                response = chat_generator.run(messages=messages, generation_kwargs={"tools": tools})

        # Regular Conversation
        else:
            messages.append(response["replies"][0])
            break
    return response["replies"][0].content


demo = gr.ChatInterface(
    fn=chatbot_with_fc,
    examples=[
        "Tell me which energy sources are present in the document?",
        "Can you describe what is the purpose of this document?",
        "What are the carbon dioxide equivalent emissions for each energy source?",
    ],
    title="Ask me about the taxonomy!",
)

demo.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


ChatMessage(content='[{"id": "call_1DhJZIlvM0VXdvHQw5ubRU1J", "function": {"arguments": "{\\"query\\":\\"Carbon dioxide equivalent emissions for each energy source\\"}", "name": "rag_pipeline_func"}, "type": "function"}]', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, meta={'model': 'gpt-3.5-turbo-0125', 'index': 0, 'finish_reason': 'tool_calls', 'usage': {'completion_tokens': 23, 'prompt_tokens': 96, 'total_tokens': 119}})


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ChatMessage(content='[{"id": "call_nTmqadiKUZibvcmIT17gqeAB", "function": {"arguments": "{\\"query\\":\\"Information about coal in the provided document\\"}", "name": "rag_pipeline_func"}, "type": "function"}]', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, meta={'model': 'gpt-3.5-turbo-0125', 'index': 0, 'finish_reason': 'tool_calls', 'usage': {'completion_tokens': 22, 'prompt_tokens': 665, 'total_tokens': 687}})


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ChatMessage(content='[{"id": "call_CrGxaINihZZHzZD9kHYMmvqp", "function": {"arguments": "{\\"query\\":\\"Conditions for nuclear energy to be considered sustainable\\"}", "name": "rag_pipeline_func"}, "type": "function"}]', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, meta={'model': 'gpt-3.5-turbo-0125', 'index': 0, 'finish_reason': 'tool_calls', 'usage': {'completion_tokens': 23, 'prompt_tokens': 739, 'total_tokens': 762}})


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ChatMessage(content='[{"id": "call_vNQSPKOHX8EELmiwsdNJtVKZ", "function": {"arguments": "{\\"query\\":\\"Conditions for natural gas to be deemed sustainable\\"}", "name": "rag_pipeline_func"}, "type": "function"}]', role=<ChatRole.ASSISTANT: 'assistant'>, name=None, meta={'model': 'gpt-3.5-turbo-0125', 'index': 0, 'finish_reason': 'tool_calls', 'usage': {'completion_tokens': 23, 'prompt_tokens': 1282, 'total_tokens': 1305}})


Batches:   0%|          | 0/1 [00:00<?, ?it/s]